<img src="img/cabecera.png?raw=1">

# **Masterclass: Streamlit-Replicate**


### Parte 2: Chatbot con Streamlit y Replicate  

Adaptado y actualizado de [*streamlit-replicate-app*](https://github.com/sfc-gh-cnantasenamat/streamlit-replicate-app)
y de [*llama2-chatbot*](https://github.com/a16z-infra/llama2-chatbot)


## Introducción

En este taller aprenderemos a crear un chatbot utilizando Streamlit como interfaz y Replicate para acceder a potentes modelos de lenguaje como Llama 3 y Claude.

### Requisitos previos
- Python 3.8 o superior
- Una cuenta en [Replicate](https://replicate.com) para obtener una API Key

### Instalación
Para empezar, crea un nuevo repositorio en tu perfil de GitHub para este proyecto y sincronízalo en tu equipo.  

Una vez creado, copia en su interior el contenido de la carpeta `proyecto-base`. Esta carpeta contiene la estructura inicial del proyecto con la configuración necesaria, incluyendo un fichero `.streamlit` con los archivos `toml` de configuración básica.  

[Extra: ¿Qué es un TOML?](https://es.wikipedia.org/wiki/TOML)

Ahora puedes crear un entorno virtual dentro de tu proyecto e instalar las dependencias necesarias:  

```bash
pip install streamlit replicate
```

### Ejecución de la aplicación
Una vez creada, podrás ejecutar tu aplicación con:

```bash
streamlit run chatbot_app.py
```

## **Paso 1: Configuración inicial**

Una vez completada la fase previa de instalación, abre el archivo `chatbot_app.py` en tu IDE favorito.  

Verás que ya contiene la configuración inicial. Si no, puedes crearlo con el siguiente código:

In [ ]:
import streamlit as st
import replicate
import os
import time

# Configuración inicial
st.set_page_config(
    page_title="Streamlit Replicate Chatbot",
    page_icon=":robot:",
    layout="wide",
    initial_sidebar_state="expanded",
)

## **Paso 2: Añadir CSS personalizado**

Podemos mejorar la apariencia de nuestra aplicación con un poco de CSS personalizado.

[Extra: ¿Qué es CSS?](https://es.wikipedia.org/wiki/CSS)

El siguiente fragmento de código nos permite:

- Ajustar el tamaño de fuente en las áreas de texto a 13px para una mejor lectura.
- Establecer ese mismo tamaño de fuente (13px) en los componentes desplegables para mayor consistencia.
- Ocultar elementos "innecesarios" como el pie de página de Streamlit y el menú principal de la aplicación.

In [ ]:
# CSS personalizado
custom_css = """
    <style>
        .stTextArea textarea {font-size: 13px;}
        div[data-baseweb="select"] > div {font-size: 13px !important;}
        footer {visibility: hidden;}
        #MainMenu {visibility: hidden;}
    </style>
"""
st.markdown(custom_css, unsafe_allow_html=True)

## **Paso 3: Definir la información de los modelos disponibles**

Crearemos un diccionario con la información de los modelos que utilizaremos, este diccionario nos permite:

- Centralizar información sobre cada modelo.
- Acceder fácilmente a los endpoints de la API de Replicate.
- Guardar metadatos importantes como: 
  - Enlaces a documentación.
  - Compatibilidad con parámetros (como top_p).
  - Requerimientos mínimos de tokens.
- Simplificar cambios entre modelos diferentes.
- Escalar la aplicación sin modificar el código principal.

In [ ]:
# Información de modelos
model_info = {
    'meta-llama-3-8b-instruct': {
        'endpoint': 'meta/meta-llama-3-8b-instruct',
        'doc_link': 'https://replicate.com/meta/meta-llama-3-8b-instruct',
        'uses_top_p': True,
        'min_tokens': 64
    },
    'meta-llama-3-70b-instruct': {
        'endpoint': 'meta/meta-llama-3-70b-instruct',
        'doc_link': 'https://replicate.com/meta/meta-llama-3-70b-instruct',
        'uses_top_p': True,
        'min_tokens': 64
    },
    'meta-llama-3.1-405b-instruct': {
        'endpoint': 'meta/meta-llama-3.1-405b-instruct',
        'doc_link': 'https://replicate.com/meta/meta-llama-3.1-405b-instruct',
        'uses_top_p': True,
        'min_tokens': 64
    },
    'meta-llama-4-17b-maverick-instruct': {
        'endpoint': 'meta/llama-4-maverick-instruct',
        'doc_link': 'https://replicate.com/meta/llama-4-maverick-instruct',
        'uses_top_p': True,
        'min_tokens': 64
    },
    'anthropic-claude-3.7-sonnet': {
        'endpoint': 'anthropic/claude-3.7-sonnet',
        'doc_link': 'https://replicate.com/anthropic/claude-3.7-sonnet',
        'uses_top_p': False,
        'min_tokens': 1024
    }
}

## **Paso 4: Inicialización de variables en el estado de la sesión**

Streamlit utiliza un estado de sesión para mantener variables entre ejecuciones. Inicializamos las variables que necesitaremos:

[Extra: Aprende más sobre Session State en Streamlit](https://docs.streamlit.io/develop/api-reference/caching-and-state/st.session_state)

In [ ]:
# Inicialización de variables
if "messages" not in st.session_state:
    st.session_state.messages = [{"role": "assistant", "content": "How may I assist you today?"}]
if "model" not in st.session_state:
    st.session_state.model = 'meta/meta-llama-3-8b-instruct'
if "selected_model" not in st.session_state:
    st.session_state.selected_model = 'meta-llama-3-8b-instruct'
if "temperature" not in st.session_state:
    st.session_state.temperature = 0.7
if "top_p" not in st.session_state:
    st.session_state.top_p = 0.9
if "max_tokens" not in st.session_state:
    st.session_state.max_tokens = 512
if "pre_prompt" not in st.session_state:
    st.session_state.pre_prompt = "You are a helpful assistant. You do not respond as 'User' or pretend to be 'User'. You only respond once as 'Assistant'."


## **Paso 5: Diseño de la barra lateral - Estructura básica y API key**

Comenzamos con la estructura básica de la barra lateral y la configuración de la API key:

In [ ]:
# Barra lateral
with st.sidebar:
    st.title('🤖 Streamlit Replicate Chatbot')
    
    # API key
    if 'REPLICATE_API_TOKEN' in st.secrets:
        st.success('API key already provided!', icon='✅')
        replicate_api = st.secrets['REPLICATE_API_TOKEN']
    else:
        replicate_api = st.text_input('Enter Replicate API token:', type='password')
        if not (replicate_api.startswith('r8_') and len(replicate_api)==40):
            st.warning('Please enter your credentials!', icon='⚠️')
        else:
            st.success('Proceed to entering your prompt message!', icon='👉')


En caso de que no hayas copiado los archivos de la carpeta `proyecto-base`, a partir de este punto para ejecutar la aplicación necesitarás crear un archivo `.streamlit/secrets.toml` con tu API key de Replicate:

```toml
REPLICATE_API_TOKEN = "tu_api_key_de_replicate"
```

Esto nos permitirá testear la aplicación durante su desarrollo, pero recuerda que **nunca debes subir tus secretos a GitHub**.  

Cuando despleguemos nuestro chatbot podremos decidir si facilitamos una API key (y asumimos los gastos del chatbot) o dejamos que los usuarios utilicen sus propias API key.

## **Paso 6: Barra lateral - Selector de modelos**

Implementamos el selector de modelos para permitir al usuario cambiar entre diferentes LLMs, recuerda que es parte de la barra lateral así que cuidado con las indentaciones:

In [ ]:
    # Selección del modelo
    st.subheader('Models and parameters')
    model_options = list(model_info.keys())
    
    selected_model = st.sidebar.selectbox(
        'Choose a model', model_options, 
        index=model_options.index(st.session_state.selected_model)
    )
    
    # Forzar recarga para aplicar cambios de modelo
    if selected_model != st.session_state.selected_model:
        st.session_state.selected_model = selected_model
        st.session_state.model = model_info[selected_model]['endpoint']
        st.rerun()
    
    current_model = st.session_state.selected_model
    current_model_info = model_info[current_model]

    # Link de documentación
    doc_link = current_model_info['doc_link']
    st.markdown(f"👉 [Learn more about this model]({doc_link}) 👈")

## **Paso 7: Barra lateral - Configuración de parámetros del modelo**

Añadimos controles deslizantes para ajustar algunos de los parámetros de generación que hemos explicado en la Parte 1, recuerda que son parte de la barra lateral así que cuidado con las indentaciones:

### Deslizador de Temperatura

In [ ]:
    # Deslizador de Temperatura
    st.session_state.temperature = st.sidebar.slider(
        'temperature', 
        min_value=0.0, 
        max_value=5.0, 
        value=st.session_state.temperature, 
        step=0.05
    )
    if st.session_state.temperature >= 1:
        st.info('Values exceeding 1 produce more creative and random outputs as well as increased likelihood of hallucination.')
    if st.session_state.temperature < 0.1:
        st.warning('Values approaching 0 produce deterministic outputs. The recommended starting value is 0.7')

### Deslizador de Top-P

In [ ]:
    # Deslizador de Top-p
    st.session_state.top_p = st.sidebar.slider(
        'top_p', 
        min_value=0.00, 
        max_value=1.0, 
        value=st.session_state.top_p, 
        step=0.05, 
        disabled=not current_model_info['uses_top_p']
    )
    if not current_model_info['uses_top_p']:
        st.warning(f'{current_model} does not use the top_p parameter.')
    else:
        if st.session_state.top_p < 0.5:
            st.warning('Low top_p values (<0.5) can make output more focused but less diverse. Recommended starting value is 0.9')
        if st.session_state.top_p == 1.0:
            st.info('A top_p value of 1.0 means no nucleus sampling is applied (considers all tokens).')

### Deslizador de Max Tokens

In [ ]:
    # Deslizador de Max tokens
    min_tokens = current_model_info['min_tokens']
    st.session_state.max_tokens = st.sidebar.slider(
        'max_length', 
        min_value=min_tokens, 
        max_value=4096, 
        value=max(min_tokens, st.session_state.max_tokens), 
        step=8
    )
    if min_tokens > 64:
        st.warning(f'{current_model} requires at least {min_tokens} input tokens.')

## **Paso 8: Barra lateral - System Prompt Editable**

Añadimos un área de texto para editar el prompt de sistema y así poder definir el comportamiento de nuestro chatbot, recuerda que es parte de la barra lateral así que cuidado con las indentaciones:

In [ ]:
    # Prompt de sistema editable
    st.subheader("System Prompt")
    new_prompt = st.text_area(
        'Edit the prompt that guides the model:',
        st.session_state.pre_prompt,
        height=100
    )
    if new_prompt != st.session_state.pre_prompt and new_prompt.strip():
        st.session_state.pre_prompt = new_prompt

## **Paso 9: Barra lateral - Botón de limpiar historial**

Implementamos un botón para limpiar el historial de chat, esto borrará los mensajes del estado de sesión de modo que "reiniciará" la conversación, recuerda que es parte de la barra lateral así que cuidado con las indentaciones:

In [ ]:
    # Botón de limpiar historial 
    def clear_chat_history():
        st.session_state.messages = [{"role": "assistant", "content": "How may I assist you today?"}]
    st.button('Clear Chat', on_click=clear_chat_history, use_container_width=True)

### *¡Por fin hemos terminado la barra lateral!*

## **Paso 10: Configuración de la API de Replicate**

Configuramos la API key para Replicate:

In [ ]:
# API token
os.environ['REPLICATE_API_TOKEN'] = replicate_api

## **Paso 11: Función para generar respuestas**

Ahora definimos la función que se comunicará con Replicate para obtener respuestas del modelo. Esta función es el corazón de nuestro chatbot y realiza varias tareas clave:

1. **Formatea el historial de conversación**:
   - Comienza con el system prompt (instrucciones iniciales para el modelo) y tras ello agrega todo el historial de mensajes con formato "User:" y "Assistant:".
   - Esto permite que el modelo tenga contexto completo de la conversación.

2. **Configura los parámetros de generación**:
   - Prepara los parámetros que controlarán el comportamiento del modelo incluyendo prompt, temperatura, longitud máxima y penalización por repetición.
   - Añade el parámetro top_p solo si el modelo seleccionado lo soporta.

3. **Transmite la respuesta en tiempo real**:
   - Utiliza la función `stream` de Replicate para obtener tokens de respuesta gradualmente e implementa un **generador** que permite mostrar la respuesta token por token.
   - Esto crea una experiencia más natural donde el usuario ve la respuesta formándose progresivamente.

In [ ]:
# Generación de respuesta
def generate_response(prompt_input):
    string_dialogue = st.session_state.pre_prompt + "\n\n"
    for dict_message in st.session_state.messages:
        if dict_message["role"] == "user":
            string_dialogue += "User: " + dict_message["content"] + "\n\n"
        else:
            string_dialogue += "Assistant: " + dict_message["content"] + "\n\n"
    
    # Parámetros base
    input_params = {
        "prompt": f"{string_dialogue}User: {prompt_input}\n\nAssistant: ",
        "temperature": st.session_state.temperature,
        "max_tokens": st.session_state.max_tokens,
        "repetition_penalty": 1,
    }
    
    # Añadir top_p solo si el modelo lo utiliza
    current_model = st.session_state.selected_model
    if model_info[current_model]['uses_top_p']:
        input_params["top_p"] = st.session_state.top_p
    
    # Stream de respuestas
    for event in replicate.stream(st.session_state.model, input=input_params):
        yield str(event)

### Extra: Generadores en Python

Los generadores son funciones especiales en Python que permiten devolver valores de manera secuencial sin tener que almacenarlos todos en memoria a la vez. Se definen usando la palabra clave `yield`.

#### Características principales

- **Lazy Execution**: Solo generan valores cuando se solicitan, ahorrando memoria.
- **Mantienen estado**: "Recuerdan" su estado entre llamadas, pausando su ejecución y retomándola donde se quedó.
- **Ideales para grandes secuencias**: Perfectos para procesar grandes conjuntos de datos de manera eficiente.
- **Iterables**: Pueden usarse en bucles for, comprensiones de listas, y otras construcciones que esperan iterables.

#### Ejemplo en nuestra aplicación

```python
# Generador que transmite respuestas del modelo
def generate_response(prompt_input):
    # Preparación de los parámetros...
    
    # Stream de respuestas usando un generador
    for event in replicate.stream(
        st.session_state.model, input=input_params
        ):
        yield str(event)

# Uso del generador para mostrar respuestas token a token
response = generate_response(prompt)
full_response = st.write_stream(response)  # Procesa cada elemento a medida que se genera
```

Este patrón nos permite mostrar la respuesta del modelo en tiempo real, token por token, creando una experiencia más fluida e interactiva para el usuario.

## **Paso 12: Interfaz de chat**

Finalmente, creamos la interfaz de chat que mostrará los mensajes y permitirá al usuario interactuar con el chatbot:

In [ ]:
# Mostrar mensajes
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.write(message["content"])

# Entrada del usuario
if prompt := st.chat_input("Type your message here...", disabled=not replicate_api):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.write(prompt)

# Generar respuesta
if st.session_state.messages and st.session_state.messages[-1]["role"] == "user":
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            response = generate_response(st.session_state.messages[-1]["content"])
            full_response = st.write_stream(response)
    st.session_state.messages.append({"role": "assistant", "content": full_response})

### Extra: Operador Morsa (Walrus Operator) en Python

El operador morsa (`:=`) fue introducido en Python 3.8 y permite asignar valores a variables como parte de una expresión. Su apodo "walrus" (morsa) viene de su apariencia visual que recuerda a los colmillos ("tusks") de este animal.

#### Usos principales

- **Asignación y evaluación en una sola expresión**: Permite asignar un valor a una variable y usarlo inmediatamente.
- **Reduce código repetitivo**: Evita cálculos o llamadas a funciones duplicadas.
- **Mejora la legibilidad**: Hace que el código sea más conciso en ciertas situaciones.

#### Ejemplo en nuestra aplicación de Streamlit

```python
# Sin operador morsa
prompt = st.chat_input("Type your message here...")
if prompt:
    st.session_state.messages.append({"role": "user", "content": prompt})

# Con operador morsa
if prompt := st.chat_input("Type your message here..."):
    st.session_state.messages.append({"role": "user", "content": prompt})
```

Este operador es especialmente útil en condicionales, bucles y comprensiones de listas donde necesitamos asignar y utilizar un valor en la misma expresión.

## **Paso 13: Configuración para despliegue**

Para desplegar tu aplicación en Streamlit Cloud:

1. Sube el código completo a tu repositorio de GitHub.
2. Conéctate a [Streamlit Cloud](https://streamlit.io/cloud).
3. Selecciona tu repositorio, la rama y el archivo principal.
4. Configura tus secretos en la interfaz de Streamlit Cloud (puedes acceder desde el menú de tres puntos de tu aplicación).
5. Es hora de desplegar tu aplicación, pero antes echa un ojo al último paso.

## **Último Paso: Añadir un sistema de autenticación (opcional)**

Si quieres proteger tu aplicación con un sistema de autenticación, puedes añadir el siguiente código justo después de las importaciones y la configuración inicial:

In [ ]:
# Comprobar autenticación
if "authenticated" not in st.session_state:
    st.session_state.authenticated = False

if not st.session_state.authenticated:
    st.title("🔐 Inicie sesión para continuar")
    
    # Obtener credenciales
    try:
        correct_username = st.secrets['USERNAME']
        correct_password = st.secrets['PASSWORD']
        
        username = st.text_input("Username")
        password = st.text_input("Password", type="password")
        
        if st.button("Login"):
            if username == correct_username and password == correct_password:
                st.session_state.authenticated = True
                st.success("Login successful!")
                time.sleep(1.0)
                st.rerun()
            else:
                st.error("Invalid username or password")
    except Exception as e:
        st.error(f"Error accessing secrets. Make sure you've set up .streamlit/secrets.toml file")

        # Bypass para desarrollo
        st.markdown("---")
        st.subheader("Opciones para desarrollo")
        st.warning("⚠️ Estas opciones solo deben usarse en entorno de desarrollo")
        
        if st.button("Continuar sin autenticación (Modo Desarrollo)", 
                    type="primary", 
                    help="Permite acceder a la aplicación sin credenciales durante desarrollo"):
            st.session_state.authenticated = True
            st.success("Accediendo en modo desarrollo...")
            time.sleep(1.0)
            st.rerun()
    
    # Detener la app si usuario no está autenticado
    st.stop()

Para usar el sistema de autenticación, deberás añadir las credenciales a tu archivo `.streamlit/secrets.toml` (**IMPORTANTE, recuerda no subir nunca tus secretos a GitHub**):

```toml
USERNAME = "admin"
PASSWORD = "password"
REPLICATE_API_TOKEN = "tu_api_key_de_replicate"
```

De manera opcional también podrás añadir un botón de logout a la barra lateral, lo puedes añadir en la barra lateral después del botón de Borrar Historial, y recuerda tener cuidado con las indentaciones:


In [ ]:
    # Botón de cerrar sesión 
    def logout():
        st.session_state.authenticated = False
    st.button('Logout', on_click=logout, use_container_width=True, type="primary")

Nota: Si bien este es un método de autenticación sencillo, puedes explorar otras formas para securizar tus aplicaciones, como por ejemplo usando plataformas de terceros más robustas como [Auth0](https://auth0.com/)

Una vez hayas añadido la autenticación, solo debes repetir el Paso 13: Configuración para el Despliegue y...

# **¡Felicidades!**  

Has creado un chatbot básico con Streamlit y Replicate que puede ser desplegado y protegido con una autenticación sencilla.